In [2]:
import math
import random

from absl import app
from absl import flags
import numpy as np

In [5]:
import sys
sys.path.append("../qcc/")
from src.lib import circuit
from src.lib import ops

In [ ]:
flags.DEFINE_integer('experiments', 1000, 'Number of experiments')
flags.DEFINE_integer('shots', 1000, 'Number of random samples')

In [7]:
angles = [0.0] * 10

In [17]:
def full_ansatz(qc):
  """The Ansatz circuit for this example."""

  qc.reg(2, 'r')

  # Step 1: Initial Rotations.
  qc.rx(0, angles[0])
  qc.rx(1, angles[1])
  qc.rz(0, angles[2])
  qc.rz(1, angles[3])

  # Step 2: Entangler.
  qc.h([0, 1])
  qc.cx(0, 1)
  qc.cx(1, 0)

  # Step 3: Final Rotations.
  qc.rz(0, angles[4])
  qc.rz(1, angles[5])
  qc.rx(0, angles[6])
  qc.rx(1, angles[7])
  qc.rz(0, angles[8])
  qc.rz(1, angles[9])


def run_two_qubit_zi_experiment():
  """Run VQE experiments with a given ansatz."""

  # Best achieved result. Goal is to get as close to +1 as possible.
  max_expect = 0.0

  # Perform experiments with randomly selected angles.
  # for experiment in range(flags.FLAGS.experiments):
  for experiment in range(1000):
    # Pick random angles.
    for i in range(10):
      angles[i] = random.random() * 2.0 * math.pi

    # Construct and run the circuit.
    qc = circuit.qc('vqe')
    full_ansatz(qc)
    qc.z(0)

    # Measure the probablities as computed from the amplitudes.
    # We only do this once per experiment.
    p0, _ = qc.measure_bit(0, collapse=True)
    p1, _ = qc.measure_bit(1, collapse=True)

    # Simulate multiple measurements by sampling over the probabilities
    # to obtain a distribution of sampled states. The measurements above
    # are the probablities that a state would be found in the |0> state.
    # For each bit, we compare this probability against another random value r.
    # If the measured probability is < r, we pretend we've actually measured an
    # |0> state, else a |1> state. We do this via sample_state() on both qubits.
    #
    def sample_state(prob_state0: float):
      if prob_state0 < random.random():
        return 1
      return 0

    num_shots = 1000
    # num_shots = flags.FLAGS.shots
    counts = [0] * 4
    for _ in range(num_shots):
      bit0 = sample_state(p0)
      bit1 = sample_state(p1)
      counts[bit1 * 2 + bit0] += 1

    # Compute the expectation value from samples measurements. Again,
    #   |00> and |01> map to Eigenvalue +1
    #   |10> and |11> map to Eigenvalue -1
    #
    # This is a bit of cheating. In this example we _know_ the
    # Eigenvalues and can therefore properly construct the expectation
    # value. I'd think in the general case it has to actually be
    # computed with <psi|H|psi>, which is still O(n^2).
    #
    expect = (counts[0] + counts[1] - counts[2] - counts[3]) / num_shots

    # Update and print currently best result.
    #
    if expect > max_expect:
      max_expect = expect
      print('Max expecation of H for experiment {:5d}: {:.4f} (target: 1.0)'.
            format(experiment, max_expect))
      print('  |00>: {}, |01>: {}, |10>: {}, |11>: {}'.format(
          counts[0], counts[1], counts[2], counts[3]))
      print('  ', end='')
      for i in range(10):
        print('{:.1f} '.format(angles[i] / 2 / math.pi * 360), end='')
      print()


def single_qubit_ansatz(theta: float, phi: float) -> circuit.qc:
  """Generate a single qubit ansatz."""

  qc = circuit.qc('single-qubit ansatz Y')
  qc.qubit(1.0)
  qc.rx(0, theta)
  qc.ry(0, phi)
  return qc


def run_single_qubit_mult():
  """Run experiments with single qubits."""

  # Construct Hamiltonian.
  hamil = (random.random() * ops.PauliX() +
           random.random() * ops.PauliY() +
           random.random() * ops.PauliZ())
  # Compute known minimum eigenvalue.
  eigvals = np.linalg.eigvalsh(hamil)

  # Brute force over the Bloch sphere.
  min_val = 1000.0
  for i in range(0, 180, 10):
    for j in range(0, 180, 10):
      theta = np.pi * i / 180.0
      phi = np.pi * j / 180.0

      # Build the ansatz with two rotation gates.
      ansatz = single_qubit_ansatz(theta, phi)

      # Compute <psi ! H ! psi>. Find smallest one, which will be
      # the best approximation to the minimum eigenvalue from above.
      # In this version, we just multiply out the result.
      psi = np.dot(ansatz.psi.adjoint(), hamil(ansatz.psi))
      if psi < min_val:
        min_val = psi

  # Result from brute force approach:
  print('Minimum: {:.4f}, Estimated: {:.4f}, Delta: {:.4f}'.format(
      eigvals[0], np.real(min_val), np.real(min_val - eigvals[0])))


def run_single_qubit_measure():
  """Run measurement experiments with single qubits."""

  # Construct Hamiltonian.
  a = random.random()
  b = random.random()
  c = random.random()
  hamil = a * ops.PauliX() + b * ops.PauliY() + c * ops.PauliZ()

  # Compute known minimum eigenvalue.
  eigvals = np.linalg.eigvalsh(hamil)

  min_val = 1000.0
  for i in range(0, 360, 5):
    for j in range(0, 180, 5):

      theta = np.pi * i / 360.0
      phi = np.pi * j / 180.0

      # X Basis
      qc = single_qubit_ansatz(theta, phi)
      qc.h(0)
      val_a = a * qc.pauli_expectation(0)

      # Y Basis
      qc = single_qubit_ansatz(theta, phi)
      qc.sdag(0)
      qc.h(0)
      val_b = b * qc.pauli_expectation(0)

      # Z Basis
      qc = single_qubit_ansatz(theta, phi)
      val_c = c * qc.pauli_expectation(0)

      expectation = val_a + val_b + val_c
      if expectation < min_val:
        min_val = expectation

  print('Minimum eigenvalue: {:.3f}, Delta: {:.3f}'
        .format(eigvals[0], min_val - eigvals[0]))

In [12]:
for _ in range(5):
    run_single_qubit_measure()
for _ in range(5):
    run_single_qubit_mult()

Minimum eigenvalue: -0.836, Delta: 0.000
Minimum eigenvalue: -1.180, Delta: 0.000
Minimum eigenvalue: -0.997, Delta: 0.000
Minimum eigenvalue: -0.344, Delta: 0.000
Minimum eigenvalue: -1.231, Delta: 0.001
Minimum: -1.0489, Estimated: -1.0473, Delta: 0.0016
Minimum: -0.9204, Estimated: -0.9182, Delta: 0.0021
Minimum: -0.9579, Estimated: -0.9547, Delta: 0.0031
Minimum: -1.0497, Estimated: -1.0486, Delta: 0.0011
Minimum: -0.9389, Estimated: -0.9287, Delta: 0.0102


In [18]:
run_two_qubit_zi_experiment()

Max expecation of H for experiment     0: 0.2640 (target: 1.0)
  |00>: 442, |01>: 190, |10>: 251, |11>: 117
  20.0 144.5 246.1 299.6 139.6 153.2 169.1 84.8 57.0 107.2 
Max expecation of H for experiment     1: 0.9480 (target: 1.0)
  |00>: 390, |01>: 584, |10>: 9, |11>: 17
  84.8 165.5 224.8 91.9 301.4 119.7 229.4 267.5 210.4 43.9 
Max expecation of H for experiment    45: 0.9860 (target: 1.0)
  |00>: 273, |01>: 720, |10>: 3, |11>: 4
  28.7 227.2 307.0 200.7 146.0 104.0 167.4 52.8 69.3 136.8 
Max expecation of H for experiment    65: 0.9880 (target: 1.0)
  |00>: 451, |01>: 543, |10>: 3, |11>: 3
  94.7 292.1 250.2 283.6 57.6 246.5 95.7 198.5 232.1 308.9 
Max expecation of H for experiment   525: 0.9920 (target: 1.0)
  |00>: 492, |01>: 504, |10>: 2, |11>: 2
  262.0 199.9 64.2 10.3 138.8 125.1 37.5 195.0 33.3 340.7 
Max expecation of H for experiment   633: 0.9980 (target: 1.0)
  |00>: 539, |01>: 460, |10>: 0, |11>: 1
  66.2 259.0 251.4 50.2 159.6 330.4 248.2 0.9 351.0 250.1 
